## Imports
utilities.py contains a number of useful functions including loading credentials from a json file, retrieving files from S3, and defining the current list of brains that are associated with the Active Atlas.

In [1]:
import datajoint as dj
import numpy as np
from minio import Minio
import json
import yaml
import sys, os

sys.path.append('./lib')
from utilities import *

## Load Credentials, Connect to Database

In [2]:
%%writefile /home/ubuntu/data/Github/VaultBrain/credFiles_aws.yaml
aws_fp: /home/ubuntu/data/Github/VaultBrain/s3-creds.json
dj_fp: /home/ubuntu/data/Github/VaultBrain/dj_local_conf.json

Writing /data/Github/VaultBrain/credFiles_aws.yaml


In [6]:
%%writefile /Users/kuiqian/Github/VaultBrain/credFiles.yaml
aws_fp: /Users/kuiqian/Github/VaultBrain/s3-creds.json
dj_fp: /Users/kuiqian/Github/VaultBrain/dj_local_conf.json

Writing /Users/kuiqian/Github/VaultBrain/credFiles.yaml


In [2]:
credFiles='/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles='/Users/kuiqian/Github/VaultBrain/credFiles.yaml'

In [3]:
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
#dj.config.save_local()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [65]:
dj.conn()

DataJoint connection (disconnected) kui@ucsd-demo-db.datajoint.io:3306

In [40]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

## Create tables

In [48]:
Structure.drop()

`kui_diffusionmap`.`structure` (28 tuples)


InternalError: (1412, 'Table definition has changed, please retry transaction')

In [42]:
@schema
class Structure(dj.Manual):
    definition = """
    structure       : char(10)    # name of mouse brain's structure
    """

In [8]:
paired_structures = ['5N', '6N', '7N', '7nn', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
all_structures = paired_structures + singular_structures

In [43]:
for struc in all_structures:
    print("\nAdding "+ struc+ " to the database")
    Structure.insert1(dict(structure=struc),
                     skip_duplicates=True)



Adding 5N to the database

Adding 6N to the database

Adding 7N to the database

Adding 7nn to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [49]:
schema.connection.close()

In [47]:
! python Patch_extractor_datajoint.py 'local' 'MD589'

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))
Connecting kui@ucsd-demo-db.datajoint.io:3306
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
224


In [46]:
Structure.fetch(as_dict=True)

[OrderedDict([('structure', '10N')]),
 OrderedDict([('structure', '12N')]),
 OrderedDict([('structure', '3N')]),
 OrderedDict([('structure', '4N')]),
 OrderedDict([('structure', '5N')]),
 OrderedDict([('structure', '6N')]),
 OrderedDict([('structure', '7N')]),
 OrderedDict([('structure', '7nn')]),
 OrderedDict([('structure', 'Amb')]),
 OrderedDict([('structure', 'AP')]),
 OrderedDict([('structure', 'DC')]),
 OrderedDict([('structure', 'IC')]),
 OrderedDict([('structure', 'LC')]),
 OrderedDict([('structure', 'LRt')]),
 OrderedDict([('structure', 'PBG')]),
 OrderedDict([('structure', 'Pn')]),
 OrderedDict([('structure', 'RMC')]),
 OrderedDict([('structure', 'RtTg')]),
 OrderedDict([('structure', 'SC')]),
 OrderedDict([('structure', 'SNC')]),
 OrderedDict([('structure', 'SNR')]),
 OrderedDict([('structure', 'Sp5C')]),
 OrderedDict([('structure', 'Sp5I')]),
 OrderedDict([('structure', 'Sp5O')]),
 OrderedDict([('structure', 'Tz')]),
 OrderedDict([('structure', 'VCA')]),
 OrderedDict([('stru

In [8]:
@schema
class Structure(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    positive_s3_fp  : varchar(200)  # (str) file path
    negative_s3_fp  : varchar(200)  # (str) file path
    """
#     positive_img_fp  : varchar(200)  # (str) file path
#     negative_img_fp  : varchar(200)  # (str) file path


In [9]:
paired_structures = ['5N', '6N', '7N', '7nn', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
all_structures = paired_structures + singular_structures
stack = 'MD589'
s3_fp = 'CSHL_regions_features/'+stack+'/' #s3://mousebrainatlas-data/
print(s3_fp)

CSHL_regions_features/MD589/


In [10]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure.insert1(dict(id=n,
                           structure=struc,
                          positive_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_positive.pkl',
                          negative_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_negative.pkl',)
                     ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7N to the database

Adding 7n to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [15]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure.insert1(dict(id=n,
                           structure=struc,
                          positive_s3_fp=s3_fp+'Properties/'+struc+'/'+stack+'_'+struc+'_positive.pkl',
                          negative_s3_fp=s3_fp+'Properties/'+struc+'/'+stack+'_'+struc+'_negative.pkl',
                          positive_img_fp=s3_fp+'Images/'+struc+'/'+stack+'_'+struc+'_positive_images.pkl',
                          negative_img_fp=s3_fp+'Images/'+struc+'/'+stack+'_'+struc+'_negative_images.pkl')
                     ,skip_duplicates=False)
    n+=1



Adding 5N to the database

Adding 6N to the database

Adding 7N to the database

Adding 7n to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [6]:
import pandas as pd
structureTable = Structure.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

NameError: name 'Structure' is not defined

In [11]:
client = get_s3_client(credFiles)

In [15]:
report=client.stat_object("mousebrainatlas-data",strucDF['positive_s3_fp'][1])
report.size

14878904

In [53]:
sys.path.append('../lib')
from utils import run

In [18]:
stack = 'MD589'
yaml_file = 'shape_params.yaml'
scripts_dir = os.environ['REPO_DIR']
struc = '5N'
state = 'positive'
run('python3 {0}/Cell_generator.py {1} {2} {3} {4}'.format(scripts_dir, stack, struc, state, yaml_file))

NameError: name 'run' is not defined

In [12]:
%%writefile test.py
print('this is me!')


Writing test.py


In [13]:
run('python test.py')

ERROR:root:File `'(python test.py).py'` not found.


In [21]:
!python test.py

this is me!


In [16]:
!cat output

this is me!
